In [42]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [43]:
data_treinamento = pd.read_csv('base_dados_treinamento.csv', sep = ';', encoding='utf-8', on_bad_lines='skip')

data_treinamento.head()

,texto,sentimento
0,@Tixaa23 14 para eu ir :),Positivo
1,@drexalvarez O meu like eu já dei na época :),Positivo
2,Eu só queria conseguir comer alguma coisa pra ...,Positivo
3,:D que lindo dia !,Positivo
4,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Positivo


In [44]:
data_treinamento['sentimento'].value_counts()

Negativo    9824
Positivo    5883
Name: sentimento, dtype: int64

In [45]:
data_aplicacao = pd.read_csv('base_dados_analise.csv', sep=';', encoding='utf-8', on_bad_lines='skip')
data_aplicacao.head()

,Feedback
0,"Eu sempre quis esse livro, eu pedi também os o..."
1,"Amo essa série, é a minha preferida, a cada li..."
2,Leitura rápida
3,Os livros de Morgan Rhodes são cheios de avent...
4,produto chegou em ótima qualidade. muito vale ...


In [46]:
#Configurando os textos de Base
def to_lower_case(texto):
  return texto.lower()

def limpar_texto(texto):
  texto = re.sub(r'http\S+', '', texto).replace('.', '')
  texto = re.sub(r'^.+@[^\.].*\.[a-z]{2,}$', 'texto-email', texto)
  texto = re.sub(r'.\$', 'valor-monetario', texto)
  texto = re.sub(r'\d(\.|,\d+)?', 'valor-numerico', texto)
  return texto


from nltk.corpus import stopwords
from string import punctuation
stopwords_custom = set(stopwords.words('portuguese') + list(punctuation))
stopwords_custom.remove('não')


def remover_stopwords(texto):
  palavras = [i for i in texto.split() if not i in stopwords_custom]
  return (" ".join(palavras))

def pre_processar_dados(texto):
  texto = to_lower_case(texto)
  texto = limpar_texto(texto)
  texto = remover_stopwords(texto)
  return texto


In [47]:
textos_treinamento = []
for texto in data_treinamento['texto']:
  #print('Antes: '+texto)
  texto_pre_processado = pre_processar_dados(texto)
  #print('Depois: '+texto_pre_processado)
  textos_treinamento.append(texto_pre_processado)

classificacao_treinamento = data_treinamento['sentimento']

In [48]:
textos_aplicacao = []
for texto in data_aplicacao['Feedback']:
  #print('Antes: '+texto)
  texto_pre_processado = pre_processar_dados(texto)
  #print('Depois: '+texto_pre_processado)
  textos_aplicacao.append(texto_pre_processado)

In [49]:
from nltk.tokenize import TweetTokenizer

text_tokenizer = TweetTokenizer()
vetorizador = CountVectorizer(analyzer='word', tokenizer=text_tokenizer.tokenize)

In [50]:
frequencia_treinamento = vetorizador.fit_transform(textos_treinamento)
#print(frequencia_treinamento)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [51]:
frequencia_aplicacao = vetorizador.transform(textos_aplicacao)
#print(frequencia_aplicacao)

In [52]:
ModeloIA = MultinomialNB()
ModeloIA.fit(frequencia_treinamento, classificacao_treinamento)

MultinomialNB()

In [53]:
modelo_aplicacao = ModeloIA.predict(frequencia_aplicacao)
textos_analizados = zip(textos_aplicacao, modelo_aplicacao)

In [54]:
resultados = []
for texto, classificacao_analizada in textos_analizados:
  resultado = {
      "Textos": texto,
      "Classificação": classificacao_analizada
  }
  resultados.append(resultado)

In [ ]:
dataframe_resultado = pd.DataFrame(resultados)
dataframe_resultado